# Pose Estimation Related to Emotions

In [1]:
!pip3 install -r requirements.txt

# Package Installation

In [2]:
import pandas as pd
from tools.segment import Segment
from tools.extractor import Extractor

# Data Preparation

Read "annotaitions.csv" file.

In [3]:
df = pd.read_csv("assets/annotations/annotations.csv")
df.head(30)

,Video Tag,Clip Id,Labels,Frame Number,X,Y,Width,Height,Person Id
0,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19532,41.965200,4.873195,44.216991,94.802684,0
1,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19538,41.564836,4.874640,44.216991,94.802684,0
2,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19544,41.164472,4.876086,44.216991,94.802684,0
3,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19550,40.764108,4.877532,44.216991,94.802684,0
4,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19556,39.646728,5.014136,44.216991,94.802684,0
5,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19562,37.784539,5.104723,44.216991,94.802684,0
6,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19568,35.908454,5.104723,44.216991,94.802684,0
7,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19574,36.047797,5.068196,44.216991,94.802684,0
8,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19580,36.590225,5.024364,44.216991,94.802684,0
9,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19586,37.132654,4.980531,44.216991,94.802684,0


Extract all frames from videos and create segments that represent a unique person.

In [4]:
extractor = Extractor(
    "assets/annotations/annotations.csv",
    "assets/videos",
    "assets/frames"
)

# extractor.extract_frames()

segments = extractor.segmentate()
for segment in segments:
    print(segment.labels_to_probs())
    print(segment)

[1. 0. 0. 0. 0. 0. 0.]
Segment (0, 27)
Video Tag: aJKL0ahn1Dk
Clip Id: 1
Person Id: 1
Size: 27

[0. 0. 1. 0. 0. 0. 0.]
Segment (28, 39)
Video Tag: aJKL0ahn1Dk
Clip Id: 1
Person Id: 2
Size: 11

[1. 0. 0. 0. 0. 0. 0.]
Segment (40, 51)
Video Tag: aJKL0ahn1Dk
Clip Id: 1
Person Id: 0
Size: 11

[0.67 0.   0.   0.33 0.   0.   0.  ]
Segment (52, 69)
Video Tag: aJKL0ahn1Dk
Clip Id: 1
Person Id: 3
Size: 17

[0.33 0.33 0.   0.33 0.   0.   0.  ]
Segment (70, 77)
Video Tag: aJKL0ahn1Dk
Clip Id: 1
Person Id: 0
Size: 7

[1. 0. 0. 0. 0. 0. 0.]
Segment (78, 89)
Video Tag: aJKL0ahn1Dk
Clip Id: 1
Person Id: 1
Size: 11

[0.33 0.   0.67 0.   0.   0.   0.  ]
Segment (90, 104)
Video Tag: aJKL0ahn1Dk
Clip Id: 1
Person Id: 2
Size: 14

[1. 0. 0. 0. 0. 0. 0.]
Segment (105, 119)
Video Tag: aJKL0ahn1Dk
Clip Id: 1
Person Id: 4
Size: 14

[0.   0.   0.33 0.67 0.   0.   0.  ]
Segment (120, 134)
Video Tag: aJKL0ahn1Dk
Clip Id: 1
Person Id: 3
Size: 14

[0.67 0.   0.   0.33 0.   0.   0.  ]
Segment (135, 149)
Video Tag: a

# Pose Detection